In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Flatten
import serial
import os
import time
from IPython import get_ipython
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [2]:
model = tf.keras.models.load_model('P1Q1_model', compile=False)

In [3]:
# Partition Layer should be the last layer before the partition
def partition(model, partitionLayer):
    num_layers = len(model.layers) - 1
    if partitionLayer >= num_layers:
        return "pick a lower partition layer, yours is or is greater than the last layer"
    clientLayers = []
    serverLayers = []
    for i in range (0,partitionLayer+1):
        clientLayers.append(model.get_layer(index=i))
        for i in range (partitionLayer+1,len(model.layers)):
            serverLayers.append(model.get_layer(index=i))
    #Creating Client and Server models
    clientModel = tf.keras.Sequential(clientLayers)
    clientModel.add(Flatten())
    serverModel = tf.keras.Sequential(serverLayers)
    #print("partitioning layer is =",partitionLayer)
    return clientModel, serverModel

In [4]:
partition_layer = 45
firstPart, lastPart = partition(model, partition_layer)

In [5]:
# wio = serial.Serial(port='COM5', baudrate=115200, timeout=1)
# def read():
#     data = wio.readline()
#     #time.sleep(0.0001)
#     return data

In [6]:
# test_output = []
# while True:
#     data_from_wio = read()
#     if data_from_wio:
#         data_from_wio = float(data_from_wio)
#         print(data_from_wio)
#         test_output.append(data_from_wio)
#         if len(test_output) == firstPart.layers[45].output.shape[1]:
#             break
# test_output = np.array(test_output)
# print(test_output.shape)
# print("Intermediate output received from the MCU!")

In [5]:
df = pd.read_csv('mcu_out.txt')
test_output = np.array(df)
#test_output = test_output.transpose()
# print(test_output)
#print(test_output.shape)

In [6]:
new_shape = lastPart.layers[0].input_shape
test_output = test_output.reshape(1, new_shape[1], new_shape[2], new_shape[3])

In [7]:
print("Starting inference...")

Starting inference...


In [10]:
inference_time = %timeit -r20 -n5 -o prediction = lastPart.predict(test_output)
# index_of_predict = np.argmax(prediction, axis=-1)
# print(index_of_predict)

24 ms ± 385 µs per loop (mean ± std. dev. of 20 runs, 5 loops each)


In [10]:
f = open('P1Q1_latency_edge.txt', 'w')
print(inference_time, file = f)
f.close()
print("Results saved!")

In [11]:
prediction = lastPart.predict(test_output)

In [12]:
index_of_predict = np.argmax(prediction, axis=-1)
print(index_of_predict)

[0]
